<br>

<div align="right" style="font-family: Arial, sans-serif; font-size: 14px; color: #333;">
    <strong>Réalisé par :</strong> ADUAYOM MESSAN Daniel  
</div>
<div align="right" style="font-family: Arial, sans-serif; font-size: 14px; color: #333;">
    <strong>Date :</strong> Octobre 2024
</div>

<br><br>

<div align="center">
    <span style="font-family: Georgia, serif; font-size: 30px; color: darkgreen;">
        PROJET DE DATA ANALYTICS
    </span>
</div>

<div align="center">
    <span style="font-family: Georgia, serif; font-size: 24px; color: #cc0000;">
        Analyse Factorielle Discriminante des Universités Françaises
    </span>
</div>

<hr style="border: 2px solid #75DFC1;">

<center>
    <h1 style="color: #444;">Étude des Similarités entre Universités</h1>
</center>

<hr style="border: 2px solid #75DFC1;">

> Ce projet s’inscrit dans une démarche d’analyse exploratoire de données sur l’insertion professionnelle des diplômés de Master en France. À travers une **Analyse Factorielle Discriminante (AFD)**, il vise à :
>
> - Comprendre les **facteurs influençant l’insertion professionnelle** selon les établissements,
> - Identifier des **profils d’universités similaires**,
> - Fournir des **indicateurs utiles aux étudiants, aux décideurs et aux établissements eux-mêmes**.
>
> L’étude repose sur des données fournies par le Ministère de l’Enseignement Supérieur (MENESR), récoltées 30 mois après l’obtention des diplômes.

<br><br>


In [5]:
# Import necessary libraries
import pandas as pd
from IPython.display import display #to display DataFrames in Jupyter notebooks

<br/>
<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Présentation de la Base de Données</h1></center>
<hr style="border-width:2px;border-color:#75DFC1">

<p style="font-family: 'Arial', sans-serif; font-size: 14px;">
    Cette base de données provient d'une enquête nationale menée par le Ministère de l’Enseignement Supérieur sur l’insertion professionnelle des diplômés de Master. Elle couvre près de 60 000 diplômés interrogés 30 mois après l’obtention de leur diplôme. Les données permettent d’analyser les performances d’insertion par université et fournissent de nombreux indicateurs socio-économiques.
</p>

<ul style="font-family: 'Arial', sans-serif; font-size: 14px;">
    <li><strong>taux_dinsertion :</strong> Pourcentage de diplômés en emploi parmi ceux présents sur le marché du travail.</li>
    <li><strong>salaire_net_median_des_emplois_a_temps_plein :</strong> Salaire mensuel net médian des diplômés en emploi à temps plein.</li>
    <li><strong>emplois_stables :</strong> Part des emplois à durée indéterminée ou statut de fonctionnaire.</li>
    <li><strong>emplois_cadre_ou_professions_intermediaires :</strong> Part des emplois de niveau cadre ou intermédiaire.</li>
    <li><strong>femmes :</strong> Pourcentage de femmes parmi les diplômés.</li>
    <li><strong>de_diplomes_boursiers :</strong> Part des diplômés ayant bénéficié d’une bourse durant leurs études.</li>
    <li><strong>emplois_exterieurs_a_la_region_de_luniversite :</strong> Part des diplômés travaillant hors de leur région universitaire.</li>
    <li><strong>taux_de_chomage_regional :</strong> Taux de chômage dans la région de l’université (source INSEE).</li>
    <li><strong>salaire_net_mensuel_median_regional :</strong> Salaire médian régional des jeunes cadres ou professions intermédiaires.</li>
    <li><strong>nombre_de_reponses :</strong> Nombre total de diplômés ayant répondu à l’enquête pour chaque université.</li>
    <li><strong>taux_de_reponse :</strong> Pourcentage de répondants à l’enquête pour chaque établissement.</li>
</ul>

<p style="font-family: 'Arial', sans-serif; font-size: 14px;">
    Ces indicateurs permettent une analyse comparative des universités françaises selon leurs résultats en matière d’insertion professionnelle. Cette base offre ainsi une opportunité précieuse pour explorer les proximités et différences entre établissements, et guider étudiants, chercheurs ou décideurs.
</p>


In [6]:
# Define the file path
file_path = 'Data/fr-esr-insertion_professionnelle-master.csv'

# Read the CSV file
# Note: The separator is set to ";" as per the original code
data = pd.read_csv(file_path,sep=";")

# Display the first few rows of the DataFrame
print("Aperçu de notre base de données")
display(data.head())

# Describe the DataFrame to get summary statistics
# Note: The describe() method provides summary statistics for numerical columns by default
print("Statistique descriptive")
display(data.describe())

# dimension of the DataFrame
print("Dimensions de notre base de données")
display((data.shape))

Aperçu de notre base de données


,annee,diplome,numero_de_l_etablissement,etablissement,etablissementactuel,code_de_l_academie,academie,code_du_domaine,domaine,code_de_la_discipline,...,taux_de_chomage_regional,salaire_net_mensuel_median_regional,emplois_cadre,emplois_exterieurs_a_la_region_de_luniversite,femmes,salaire_net_mensuel_regional_1er_quartile,salaire_net_mensuel_regional_3eme_quartile,cle_etab,cle_disc,id_paysage
0,2017,MASTER LMD,0751719L,Paris 3 - Sorbonne Nouvelle,NaN,A01,Paris,LLA,"Lettres, langues, arts",disc06,...,7.1,2120,46,35,89,1620,2620,Paris 3 - Sorbonne Nouvelle_18,disc06_18,8k883
1,2017,MASTER LMD,0751719L,Paris 3 - Sorbonne Nouvelle,NaN,A01,Paris,LLA,"Lettres, langues, arts",disc06,...,7.1,2120,57,35,89,1620,2620,Paris 3 - Sorbonne Nouvelle_30,disc06_30,8k883
2,2017,MASTER ENS,0755890V,Sorbonne Université,NaN,A01,Paris,MEEF,Masters enseignement,disc18,...,7.1,2120,96,6,86,1620,2620,Sorbonne Université_30,disc18_30,bxPQe
3,2017,MASTER LMD,0755890V,Sorbonne Université,NaN,A01,Paris,DEG,"Droit, économie et gestion",disc01,...,7.1,2120,ns,ns,ns,1620,2620,Sorbonne Université_30,disc01_30,bxPQe
4,2017,MASTER LMD,0755890V,Sorbonne Université,NaN,A01,Paris,STS,"Sciences, technologies et santé",disc16,...,7.1,2120,93,15,9,1620,2620,Sorbonne Université_18,disc16_18,bxPQe


Statistique descriptive


,annee,nombre_de_reponses
count,19603.000000,19556.000000
mean,2015.412692,128.106770
std,2.960641,629.049058
min,2010.000000,0.000000
25%,2013.000000,14.000000
50%,2015.000000,35.000000
75%,2018.000000,83.000000
max,2020.000000,13587.000000


Dimensions de notre base de données


(19603, 35)